In [ ]:
import pandas as pd
import xarray as xr
import os

import numpy as np
import gsw
import warnings
import matplotlib.pyplot as plt
from erddapy import ERDDAP

# !pip install cmocean
from netCDF4 import Dataset
import matplotlib.dates as mdates

import geopy.distance
import gsw
import matplotlib.ticker as mticker
import cmocean
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
e = ERDDAP(server="https://gliders.ioos.us/erddap")
e.constraints = None
e.protocol = "tabledap"
e.dataset_id = "ce_383-20220720T1508"

opendap_url = e.get_download_url(
    response="opendap",
)

print(opendap_url)

In [ ]:
e.response = "nc"
e.variables = ["longitude","depth","latitude","backscatter", "temperature", "salinity","time","dissolved_oxygen","chlorophyll"]

df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True,
).dropna()

df.head()

In [ ]:
start = '2022-08-01 00:00:00+00:00'
end = '2022-08-11 19:00:00+00:00'
df = df.loc[start:end,:]
df.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(17, 2))
cs = ax.scatter(
    df.index,
    df["depth (m)"],
    s=15,
    c=df["temperature (Celsius)"],
    marker="o",
    edgecolor="none"
)

ax.invert_yaxis()
ax.set_xlim(df.index[0], df.index[-1])
xfmt = mdates.DateFormatter("%H:%Mh\n%d-%b")
ax.xaxis.set_major_formatter(xfmt)

cbar = fig.colorbar(cs, orientation="vertical", extend="both")
cbar.ax.set_ylabel("Temperature ($^\circ$C)")
ax.set_ylabel("Depth (m)");

In [ ]:
fig, ax = plt.subplots()

ax.plot(df.index, df["depth (m)"]);
ax.set_ylabel("Depth (m)");
ax.invert_yaxis()
ax.set_xlim(df.index[0], df.index[-1])
xfmt = mdates.DateFormatter("%H:%Mh\n%d-%b")
ax.xaxis.set_major_formatter(xfmt)
fig.autofmt_xdate()

In [ ]:
df['distance']=df.apply(lambda row: geopy.distance.distance((47, -124.173),(row['latitude (degrees_north)'],row['longitude (degrees_east)'])).km, axis=1)

In [ ]:
df['distance']

In [ ]:
# Plot the data - Distance on x-axis
fig, (ax1,ax2,ax3) = plt.subplots(3, sharex=True, sharey=True)
fig.set_size_inches(6.4,4.8)
# fig.set_size_inches(8,8)
fig.set_dpi(150)

ms = 56 #marker size
a1 = ax1.scatter(df.distance, df['depth (m)'], c=df['temperature (Celsius)'], marker=".", s=ms, cmap =cmocean.cm.haline)
a2 = ax2.scatter(df.distance, df['depth (m)'], c=df['salinity (1)'], marker=".", s=ms, cmap =cmocean.cm.haline, vmin=32, vmax=34.2)
a3 = ax3.scatter(df.distance, df['depth (m)'], c=df['dissolved_oxygen (umol kg-1)'], marker=".", s=ms, cmap =cmocean.cm.haline) #vmin=0, vmax=300

cb1 = plt.colorbar(a1,ax=ax1, aspect=10)
cb2 = plt.colorbar(a2,ax=ax2,extend='min', aspect=10)
cb3 = plt.colorbar(a3,ax=ax3, aspect=10)
cb1.set_label('Temperature (°C)',fontsize='x-small')
cb2.set_label('Salinity (psu)',fontsize='x-small')
cb3.set_label('Dissolved Oxygen (uM/kg)',fontsize='x-small')

ax2.set_ylabel("Depth (m)",fontsize='x-small')
ax1.invert_yaxis()

ax3.set_xlabel('Distance from lat/lon of coast at 47N latitude (glider line)',fontsize='x-small')
ax1.set_xlim(min(df.distance),max(df.distance));
ax1.invert_xaxis()

ax1.set_title('Glider #320')

ax1.tick_params(labelsize='xx-small')
ax2.tick_params(labelsize='xx-small')
ax3.tick_params(labelsize='xx-small')
cb1.ax.tick_params(labelsize='xx-small')
cb2.ax.tick_params(labelsize='xx-small')
cb3.ax.tick_params(labelsize='xx-small')

In [ ]:
plt.figure(figsize=(8,6))

plt.plot(df['longitude (degrees_east)'], df['latitude (degrees_north)'], c='black')
plt.scatter(df['longitude (degrees_east)'], df['latitude (degrees_north)'], c=df.index, cmap='RdYlBu')

# Label Start/End Points
plt.text(df['longitude (degrees_east)'].iloc[0], df['latitude (degrees_north)'].iloc[0], 'Start')
plt.text(df['longitude (degrees_east)'].iloc[-1], df['latitude (degrees_north)'].iloc[-1], 'End')

# Add a colorbar
cbar = plt.colorbar();

# Fix the colorbar ticks
ticks_loc=cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%Y-%m-%d'));
ax3.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))

In [ ]:
df['latitude (degrees_north)'].iloc[-1]

In [ ]:
# Plot the data - Distance on x-axis
fig, (ax1,ax2,ax3) = plt.subplots(3, sharex=True, sharey=True)
fig.set_size_inches(6.4,4.8)
# fig.set_size_inches(8,8)
fig.set_dpi(150)

ms = 56 #marker size
a1 = ax1.scatter(df.index, df['depth (m)'], c=df['temperature (Celsius)'], marker=".", s=ms, cmap =cmocean.cm.thermal)
a2 = ax2.scatter(df.index, df['depth (m)'], c=df['salinity (1)'], marker=".", s=ms, cmap=cmocean.cm.thermal, vmin=32, vmax=34.2)
a3 = ax3.scatter(df.index, df['depth (m)'], c=df['dissolved_oxygen (umol kg-1)'], marker=".", s=ms, cmap=cmocean.cm.thermal) #vmin=0, vmax=300


cb1 = plt.colorbar(a1,ax=ax1, aspect=10)
cb2 = plt.colorbar(a2,ax=ax2,extend='min', aspect=10)
cb3 = plt.colorbar(a3,ax=ax3, aspect=10)
cb1.set_label('Temperature (°C)',fontsize='x-small')
cb2.set_label('Salinity (psu)',fontsize='x-small')
cb3.set_label('Dissolved Oxygen (uM/kg)',fontsize='x-small')

ax2.set_ylabel("Depth (m)",fontsize='x-small')
ax1.invert_yaxis()

ax1.set_xlim(min(df.index),max(df.index));
ax1.invert_xaxis()

ax1.set_title('Glider #320')

ax1.tick_params(labelsize='xx-small')
ax2.tick_params(labelsize='xx-small')
ax3.tick_params(labelsize='xx-small')
cb1.ax.tick_params(labelsize='xx-small')
cb2.ax.tick_params(labelsize='xx-small')
cb3.ax.tick_params(labelsize='xx-small')
ax1.xaxis.set_major_formatter(xfmt)
fig.autofmt_xdate()

In [ ]:
# Temperature Salinity Profile with Date
fig,(ax1,ax2) = plt.subplots(1, 2, sharey=True)
dx = ax1.scatter(df['temperature (Celsius)'],  df['depth (m)'], c=df.index, s=2)
dx1 = ax2.scatter(df['salinity (1)'],  df['depth (m)'], c=df.index, s=2)
ax1.invert_yaxis()

ax1.set_xlabel('Temperature  (°C)')
ax2.set_xlabel('Salinity (psu)')
ax1.set_ylabel('Depth (m)');

cb =fig.colorbar(dx1,ax=ax2)
cb.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%Y-%m-%d'));

In [ ]:
smin = df['salinity (1)'].min() - (0.01* df['salinity (1)'].min())
smax = df['salinity (1)'].max() + (0.01 * df['salinity (1)'].max())
tmin = df['temperature (Celsius)'].min() - (0.1 * df['temperature (Celsius)'].max())
tmax = df['temperature (Celsius)'].max() + (0.1 * df['temperature (Celsius)'].max())

# Calculate the density lines
x = int(round((smax-smin)/0.1+1,0))
y = int(round((tmax-tmin)+1,0))

In [ ]:
# Create temp and salt vectors of appropiate dimensions
ti = np.linspace(1,y-1,y)+tmin
si = np.linspace(1,x-1,x)*0.1+smin

In [ ]:
# Create empty grid of zeros
dens = np.zeros((y,x))

# Loop to fill in grid with densities
for j in range(0,int(y)):
    for i in range(0, int(x)):
        dens[j,i]=float(gsw.rho(si[i],ti[j],0))
        

In [ ]:
# Substract 1000 to convert to sigma-t
dens = dens - 1000

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
CS = plt.contour(si,ti,dens, linestyles='dashed', colors='k')
plt.clabel(CS, fontsize=12, inline=1, fmt='%.1f') # Label every second level
 
sc1 =ax1.scatter(df['salinity (1)'],df['temperature (Celsius)'], c=df['dissolved_oxygen (umol kg-1)'], s=2)
 
ax1.set_xlabel('Salinity (psu)')
ax1.set_ylabel('Temperature  (°C)')
cb1 = plt.colorbar(sc1, ax=ax1, aspect=10, label='Dissolved Oxygen (uM/kg)')
plt.show()

In [ ]:
spice = np.zeros((y,x))
# Loop to fill in grid with densities
for j in range(0,int(y)):
    for i in range(0, int(x)):
        spice[j,i]=float(gsw.spiciness0(si[i],ti[j]))

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
C1= plt.contour(si,ti,dens, linestyles='dashed', colors='k')
C2 = plt.contour(si,ti,spice, linestyles='-', colors='red')
plt.clabel(C1, fontsize=12, inline=1, fmt='%.1f') # Label every second level
plt.clabel(C2, fontsize=12, inline=1, fmt='%.1f') # Label every second level

sc1 =ax1.scatter(df['salinity (1)'],df['temperature (Celsius)'], c=df['dissolved_oxygen (umol kg-1)'], s=2)
 
ax1.set_xlabel('Salinity (psu)')
ax1.set_ylabel('Temperature  (°C)')
cb1 = plt.colorbar(sc1, ax=ax1, aspect=10, label='Dissolved Oxygen (uM/kg)')
plt.show()